## 1. Import Library yang dibutuhkan

Konfigurasi perangkat (menggunakan GPU jika tersedia, jika tidak menggunakan CPU)

In [5]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('isAvailable: ', device)

isAvailable:  cuda


In [6]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


## 2. Hyperparameter

In [7]:
batch_size = 64
learning_rate = 0.001
num_epochs = 100

## 3. Transformasi dan Normalisasi data

In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalisasi data MNIST
])

## 4. Load dataset MNIST

In [9]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Masukkan dataset ke dalam DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## 5.  Membangun model Convolutional Neural Networks (CNN)


In [10]:
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.bn1 = nn.BatchNorm1d(512)  # Batch normalization setelah fully connected layer 1
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)  # Batch normalization setelah fully connected layer 2
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)  # Batch normalization setelah fully connected layer 3
        self.fc4 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.4)  # Dropout untuk mencegah overfitting

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Meratakan input gambar menjadi vektor 1 dimensi
        x = F.relu(self.bn1(self.fc1(x)))  # Aktivasi ReLU setelah batch norm layer 1
        x = self.dropout(x)  # Dropout
        x = F.relu(self.bn2(self.fc2(x)))  # Aktivasi ReLU setelah batch norm layer 2
        x = self.dropout(x)  # Dropout
        x = F.relu(self.bn3(self.fc3(x)))  # Aktivasi ReLU setelah batch norm layer 3
        x = self.dropout(x)  # Dropout
        x = self.fc4(x)  # Tidak menggunakan aktivasi untuk output, karena CrossEntropyLoss sudah memasukkan softmax
        return x

## 6. Inisialisasi Model, Loss Function dan Optimizer

In [11]:
model = MNISTModel()
criterion = nn.CrossEntropyLoss()  # Fungsi loss untuk klasifikasi multi-kelas
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Optimizer Adam

## 7. Fungsi Training

In [12]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()  # Set model ke mode training
    for epoch in range(num_epochs):
        total_loss = 0
        for images, labels in train_loader:
            optimizer.zero_grad()  # Reset gradien
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Hitung loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update bobot

            total_loss += loss.item()  # Simpan total loss

        avg_loss = total_loss / len(train_loader)  # Rata-rata loss per epoch
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

## 8. Fungsi Testing

In [13]:
def test_model(model, test_loader):
    model.eval()  # Set model ke mode evaluasi
    correct = 0
    total = 0
    with torch.no_grad():  # Tidak perlu hitung gradien saat testing
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  # Prediksi label dengan probabilitas tertinggi
            total += labels.size(0)  # Total data
            correct += (predicted == labels).sum().item()  # Hitung prediksi yang benar

    accuracy = 100 * correct / total  # Hitung akurasi
    print(f'Akurasi model pada 10.000 gambar test: {accuracy:.2f}%')

## 9. Evaluasi dan Validasi Model

In [14]:
train_model(model, train_loader, criterion, optimizer, num_epochs)
test_model(model, test_loader)

Epoch [1/100], Loss: 0.3507
Epoch [2/100], Loss: 0.1818
Epoch [3/100], Loss: 0.1467
Epoch [4/100], Loss: 0.1233
Epoch [5/100], Loss: 0.1157
Epoch [6/100], Loss: 0.1018
Epoch [7/100], Loss: 0.0963
Epoch [8/100], Loss: 0.0868
Epoch [9/100], Loss: 0.0826
Epoch [10/100], Loss: 0.0775
Epoch [11/100], Loss: 0.0718
Epoch [12/100], Loss: 0.0700
Epoch [13/100], Loss: 0.0648
Epoch [14/100], Loss: 0.0607
Epoch [15/100], Loss: 0.0596
Epoch [16/100], Loss: 0.0575
Epoch [17/100], Loss: 0.0543
Epoch [18/100], Loss: 0.0508
Epoch [19/100], Loss: 0.0509
Epoch [20/100], Loss: 0.0473
Epoch [21/100], Loss: 0.0448
Epoch [22/100], Loss: 0.0458
Epoch [23/100], Loss: 0.0441
Epoch [24/100], Loss: 0.0407
Epoch [25/100], Loss: 0.0425
Epoch [26/100], Loss: 0.0419
Epoch [27/100], Loss: 0.0363
Epoch [28/100], Loss: 0.0392
Epoch [29/100], Loss: 0.0373
Epoch [30/100], Loss: 0.0348
Epoch [31/100], Loss: 0.0348
Epoch [32/100], Loss: 0.0358
Epoch [33/100], Loss: 0.0342
Epoch [34/100], Loss: 0.0321
Epoch [35/100], Loss: 0